In [16]:
import arrow
import socket
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

from april import Evaluator
from april.anomalydetection import *
from april.database import EventLog
from april.database import Model
from april.database import get_engine
from april.dataset import Dataset
from april.fs import DATE_FORMAT
from april.fs import get_event_log_files
dataset_name = 'p2p-0.3-1'

In [17]:
import gzip
import json
import pandas as pd
# def load_binet_data (path):
#     with gzip.open(path, "r") as f:
#         data = f.read()
#         j = json.loads (data.decode('utf-8'))
        
#     event_df = pd.DataFrame({'attributes': [], 'name': [], 'timestamp': [],'timestamp_end': [], 'event_id': [], 'trace_id': []})
#     truth_df = pd.DataFrame({'case': [], 'anomalies': []})
#     res=[]
#     res=pd.DataFrame()
#     for case in (j['cases']):
#         trace = pd.DataFrame.from_dict(case['events'])
#         trace['anomaly']= case['attributes']['label'] if type(case['attributes']['label'])==str else case['attributes']['label']['anomaly']
#         trace['trace_id']=case['id']
#         res=res.append(trace)
#     res=pd.concat([res.drop(['attributes'], axis=1), res['attributes'].apply(pd.Series)], axis=1)
#     return res

In [18]:
def load_binet_data(path):
    with gzip.open(path, "r") as f:
        data = f.read()
        j = json.loads(data.decode('utf-8'))
    
    res_list = []
    
    for case in j['cases']:
        trace = pd.DataFrame.from_dict(case['events'])
        trace['anomaly'] = case['attributes']['label'] if isinstance(case['attributes']['label'], str) else case['attributes']['label']['anomaly']
        trace['trace_id'] = case['id']
        res_list.append(trace)
    
    if res_list:
        res = pd.concat(res_list, ignore_index=True)
        res = pd.concat([res.drop(['attributes'], axis=1), res['attributes'].apply(pd.Series)], axis=1)
    else:
        res = pd.DataFrame()
    
    return res

In [19]:
# path_to_dataset = '/mnt/d/binet/.out/eventlogs/p2p-0.3-1.json.gz'
path_to_dataset = r"D:\binet\.out\eventlogs\p2p-0.3-1.json.gz"

In [20]:
dataset_df = load_binet_data(path_to_dataset)
dataset = Dataset(dataset_name)


In [21]:
flat_onehot_features_2d = dataset.flat_onehot_features_2d
flat_features = dataset.flat_features
features  = dataset.features
print(f"features.shape: {len(features)}")
print(f"flat_features.shape: {flat_features.shape}")
print(f"flat_onehot_features_2d.shape: {flat_onehot_features_2d.shape}")

features.shape: 2
flat_features.shape: (5000, 16, 2)
flat_onehot_features_2d.shape: (5000, 2688)


In [22]:
features[1].shape

(5000, 16)

In [ ]:
print(features[0][0])
print(features[1][0])
# print(features[2][0])
print(flat_features[0])
print(type(flat_features))



[27.  7. 11.  4.  5. 24.  9. 10.  8. 26.  0.  0.  0.  0.  0.  0.]
[141.  32.  51.  27. 121.  13.  49. 123.  40. 140.   0.   0.   0.   0.
   0.   0.]
[[ 27. 141.]
 [  7.  32.]
 [ 11.  51.]
 [  4.  27.]
 [  5. 121.]
 [ 24.  13.]
 [  9.  49.]
 [ 10. 123.]
 [  8.  40.]
 [ 26. 140.]
 [  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [  0.   0.]]
<class 'numpy.ndarray'>
[0. 0. 0. ... 0. 0. 0.]


In [25]:
print(flat_onehot_features_2d.shape)


(5000, 2688)


In [ ]:
from pprint import pprint
def dump(obj):
  for attr in dir(obj):
    pprint("obj.%s = %r" % (attr, getattr(obj, attr)))
dump(dataset)

"obj.__class__ = <class 'april.dataset.Dataset'>"
("obj.__delattr__ = <method-wrapper '__delattr__' of Dataset object at "
 '0x7f757fd7f970>')
("obj.__dict__ = {'dataset_name': 'p2p-0.3-1', 'go_backwards': False, "
 "'pad_mode': 'post', 'attribute_types': [<AttributeType.CATEGORICAL: 0>, "
 "<AttributeType.CATEGORICAL: 0>], 'attribute_keys': ['name', 'user'], "
 "'classes': array([[[0, 0],\n"
 '        [0, 0],\n'
 '        [0, 0],\n'
 '        ...,\n'
 '        [0, 0],\n'
 '        [0, 0],\n'
 '        [0, 0]],\n'
 '\n'
 '       [[0, 0],\n'
 '        [0, 0],\n'
 '        [0, 0],\n'
 '        ...,\n'
 '        [0, 0],\n'
 '        [0, 0],\n'
 '        [0, 0]],\n'
 '\n'
 '       [[0, 0],\n'
 '        [0, 0],\n'
 '        [0, 0],\n'
 '        ...,\n'
 '        [0, 0],\n'
 '        [0, 0],\n'
 '        [0, 0]],\n'
 '\n'
 '       ...,\n'
 '\n'
 '       [[0, 0],\n'
 '        [0, 0],\n'
 '        [0, 0],\n'
 '        ...,\n'
 '        [0, 0],\n'
 '        [0, 0],\n'
 '        [0, 0]],\n'
 '\n

In [ ]:
# pprint(dataset.attribute_dims)
# dataset.encoders['name'].inverse_transform(features[0][0])
# sign = dataset.encoders['user'].classes_[-1]
sign = "Create SC"
# pprint(dataset.encoders['user'].classes_)
pprint(features[0][0])
pprint(dataset.encoders['name'].transform([sign]))


array([27.,  7., 11.,  4.,  5., 24.,  9., 10.,  8., 26.,  0.,  0.,  0.,
        0.,  0.,  0.])
array([6])
